In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import dates
import seaborn as sns

from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from nltk import FreqDist, word_tokenize, RegexpTokenizer, stem
from snowballstemmer import TurkishStemmer

import time
from string import punctuation
from pathlib import Path


In [2]:
# file_name = "Recommendation_large_STATE.xlsx"
# file_name = "worldwide-aggregate.xlsx"
file_name = "titanic.xlsx"
datasets = Path().cwd().parent.parent.parent / "Datasets"
df = pd.read_excel(datasets / file_name)
column_types = {"CLIENTNAME":"categoric", "BRAND":"categoric", "CATEGORY_NAME3":"categoric", "timestamp":"datetime", "PRICE":"numeric"}
# df.loc[:, "Increase rate"] = df.loc[:,"Increase rate"].apply(lambda x: x/1e14)
df.head()

,Survived,Pclass,Name,Sex,Age,Fare,TIMESTAMP,Date,text
0,0.0,1.0,Capt. Edward Gifford Crosby,male,70.0,71.0,889069782.0,2022-02-01,chico ilimitado your service is great. I will ...
1,0.0,1.0,Col. John Weir,male,60.0,2655.0,884129542.0,2021-12-13,virginmobilemx very good service
2,1.0,1.0,Col. Oberst Alfons Simonius-Blumer,male,56.0,355.0,880604077.0,2021-10-27,virginmobilemx service quality is awesome
3,0.0,1.0,Don. Manuel E Uruchurtu,male,40.0,277208.0,882075827.0,2020-02-21,@DilanKurtt1 Peki bu kızın ayda kutay olması :(
4,0.0,2.0,Dr. Alfred Pain,male,23.0,105.0,891447773.0,2021-02-21,"uygulamanız güzel ama Salı uygulaması yok, get..."


In [3]:
df.dtypes

Survived            float64
Pclass              float64
Name                 object
Sex                  object
Age                 float64
Fare                float64
TIMESTAMP           float64
Date         datetime64[ns]
text                 object
dtype: object

In [47]:
def cast_datatypes(df, user_selected_types):
    types = user_selected_types.get("types")
    types_for_pandas = {}
    df_copy = df.copy().dropna()
    for col_name, col_type in types.items():
        if col_type == "text":
            try:
                df_copy.loc[:, col_name] = df_copy.loc[:, col_name].astype(dtype={col_name:"object"})
            except Exception as error:
                raise TypeError("Could not convert {name} column to {dtype}. Detail: {err}".format(name=col_name, dtype=col_type, err=error))
        elif col_type == "categoric":
            try:
                df_copy.loc[:, col_name] = df_copy.loc[:, col_name].astype(dtype={col_name:"category"})
            except Exception as error:
                raise TypeError("Could not convert {name} column to {dtype}. Detail: {err}".format(name=col_name, dtype=col_type, err=error))
        elif col_type == "numeric":
            try:
                df_copy.loc[:, col_name] = df_copy.loc[:, col_name].astype(dtype={col_name:"int64"})
            except Exception as error:
                raise TypeError("Could not convert {name} column to {dtype}. Detail: {err}".format(name=col_name, dtype=col_type, err=error))
        elif col_type == "datetime":
            try:
                df_copy.loc[:, col_name] = df_copy.loc[:, col_name].astype(dtype={col_name:"datetime64[s]"})
            except Exception as error:
                raise TypeError("Could not convert {name} column to {dtype}. Detail: {err}".format(name=col_name, dtype=col_type, err=error))

    # df_copy = df.copy().dropna().astype(dtype=types_for_pandas)
    return df_copy

In [50]:
user_selected_types = {"types": {"Survived": "text", "Pclass": "text", "Name": "text", "Sex": "text", "Age": "text", "Fare": "text", "TIMESTAMP": "text", "Date": "text", "text": "numeric"}, "file_name": "titanic.xlsx"}
user_selected_types

{'types': {'Survived': 'text',
  'Pclass': 'text',
  'Name': 'text',
  'Sex': 'text',
  'Age': 'text',
  'Fare': 'text',
  'TIMESTAMP': 'text',
  'Date': 'text',
  'text': 'numeric'},
 'file_name': 'titanic.xlsx'}

In [51]:
df_copy = cast_datatypes(df, user_selected_types)

TypeError: Could not convert text column to numeric. Detail: invalid literal for int() with base 10: 'chico ilimitado your service is great. I will advise it to my friends.'

In [11]:
df_copy.dtypes

Survived             object
Pclass               object
Name                 object
Sex                  object
Age                  object
Fare                 object
TIMESTAMP            object
Date         datetime64[ns]
text                 object
dtype: object

In [51]:
df_copy.dtypes

Survived             object
Pclass               object
Name                 object
Sex                  object
Age                  object
Fare                 object
TIMESTAMP            object
Date         datetime64[ns]
text                 object
dtype: object